# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:

* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connect to the Database

Include the relevant imports, then connect to the database located at `data.sqlite`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor

In [3]:
pd.read_sql(""" SELECT name FROM sqlite_master 
            WHERE type = 'table';""", conn)

,name
0,productlines
1,offices
2,customers
3,employees
4,orderdetails
5,orders
6,payments
7,products


## Employees and Their Offices (a One-to-One Join)

Select all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [10]:
pd.read_sql(""" SELECT e.firstName, e.lastName, o.city, o.state
            FROM employees e
            JOIN offices o
            ON e.officeCode = o.officeCode
            ORDER BY firstName, lastName;""", conn
            )

,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and Their Orders (a One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [11]:
pd.read_sql("""
            SELECT c.contactFirstName, c.contactLastName, o.orderNumber, o.orderDate, o.status
            FROM customers c
            JOIN orders o
            USING(customerNumber)
            GROUP BY c.contactFirstName;""", conn
            )

,contactFirstName,contactLastName,orderNumber,orderDate,status
0,Adrian,Huxley,10139,2003-07-16,Shipped
1,Akiko,Shimamura,10258,2004-06-15,Shipped
2,Allen,Nelson,10147,2003-09-05,Shipped
3,Ann,Brown,10129,2003-06-12,Shipped
4,Anna,O'Hara,10148,2003-09-11,Shipped
...,...,...,...,...,...
83,Wendy,Victorino,10217,2004-02-04,Shipped
84,William,Brown,10233,2004-03-29,Shipped
85,Wing,Huang,10143,2003-08-10,Shipped
86,Yoshi,Tamuri,10206,2003-12-05,Shipped


## Customers and Their Payments (Another One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [12]:
pd.read_sql("""
            SELECT c.contactFirstName, c.contactLastName, p.amount, p.paymentDate
            FROM customers c
            JOIN payments p
            USING(customerNumber)
            GROUP BY c.contactFirstName, c.contactLastName
            ORDER BY p.amount DESC;""", conn
            )

,contactFirstName,contactLastName,amount,paymentDate
0,Julie,Brown,85559.12,2003-11-03
1,Veysel,Oeztan,85024.46,2003-12-03
2,Valarie,Thompson,80375.24,2004-03-15
3,Kelvin,Leong,63357.13,2004-09-07
4,Sue,Frick,59551.38,2004-06-21
...,...,...,...,...
93,Mary,Young,4465.85,2003-10-18
94,Eduardo,Saavedra,3101.40,2003-05-09
95,Mike,Graham,2880.00,2003-07-05
96,Yu,Choi,1679.92,2004-04-30


## Orders, Order Details, and Product Details (a Many-to-Many Join)

Select all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [13]:
pd.read_sql("""
            SELECT c.contactFirstName, c.contactLastName, p.productName, od.quantityOrdered, o.orderDate
            FROM customers c
            JOIN orders o
            USING(customerNumber)
            JOIN orderdetails od
            USING(orderNumber)
            JOIN products p
            USING(productCode)
            GROUP BY c.contactFirstName, c.contactLastName
            ORDER BY o.orderDate DESC;""", conn
            )

,contactFirstName,contactLastName,productName,quantityOrdered,orderDate
0,Michael,Donnermeyer,1980s Black Hawk Helicopter,36,2004-09-15
1,Marta,Hernandez,1969 Harley Davidson Ultimate Chopper,36,2004-08-27
2,Jan,Klaeboe,1980s Black Hawk Helicopter,45,2004-08-21
3,Paolo,Accorti,1952 Alpine Renault 1300,34,2004-08-17
4,Miguel,Barajas,1968 Ford Mustang,50,2004-08-02
...,...,...,...,...,...
93,Diego,Freyre,1969 Corvair Monza,34,2003-01-31
94,Jonas,Bergulfsen,1952 Alpine Renault 1300,26,2003-01-29
95,Michael,Frick,1937 Lincoln Berline,39,2003-01-10
96,Roland,Keitel,1932 Model A Ford J-Coupe,25,2003-01-09


## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!